# Structured Output with Pydantic Models

This notebook demonstrates how to get structured, validated data from agent responses using Pydantic models instead of free-form text.

## Key Concepts
- **Pydantic Models**: Define expected data structure
- **Type Validation**: Automatic validation and error checking
- **Structured Responses**: Guaranteed data format for downstream processing
- **Database Integration**: Easy integration with databases and APIs

## Benefits
- **Guaranteed Format**: Consistent output regardless of model variations
- **Type Safety**: Validation prevents data errors
- **Easy Processing**: Direct use in applications without parsing
- **Documentation**: Self-documenting data structures

## Prerequisites

Make sure you have the required packages installed:

```bash
pip install langchain langchain-community langchain-core langgraph pydantic
ollama pull qwen3
ollama serve
```

In [2]:
# Import required modules
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional, Union
from datetime import datetime
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
import tools

## Basic Structured Output Example

Let's start with a simple contact information extraction example:

In [4]:
print("=== Basic Structured Output Example ===")

# Define the structure we want the agent to return
class ContactInfo(BaseModel):
    """Contact information structure."""
    name: str = Field(description="Full name of the person")
    email: str = Field(description="Email address")
    phone: str = Field(description="Phone number")
    company: str = Field(default="Unknown", description="Company name (optional)")
    
    @field_validator('email')
    def validate_email(cls, v):
        """Basic email validation."""
        if '@' not in v:
            raise ValueError('Invalid email format')
        return v

# Display the model structure
print("✓ ContactInfo model defined with fields:")
for field_name, field_info in ContactInfo.model_fields.items():
    required = "(required)" if field_info.is_required() else "(optional)"
    print(f"  - {field_name}: {field_info} {required}")
    if field_info.field_info.description:
        print(f"    Description: {field_info.field_info.description}")

=== Basic Structured Output Example ===
✓ ContactInfo model defined with fields:
  - name: annotation=str required=True description='Full name of the person' (required)


AttributeError: 'FieldInfo' object has no attribute 'field_info'

## Creating Agent with Structured Output

Note: In this demo, we'll show the concept. Full structured output integration depends on your LangChain version and model support.

In [ ]:
# Create model and agent
model = ChatOllama(model="qwen3")

# Create agent with structured output requirement (conceptual)
# Note: Actual implementation may vary based on LangChain version
agent = create_agent(
    model,
    tools=[tools.extract_contact],
    # response_format=ContactInfo  # This would force structured output in some implementations
)

print("✓ Agent created for contact information extraction")
print("  The agent will extract contact information and format it consistently")

## Testing Contact Information Extraction

In [ ]:
# Test with contact information
contact_text = "Extract contact info from: John Doe, john@example.com, (555) 123-4567, works at TechCorp"

print(f"Input text: {contact_text}")
print("\n=== Agent Processing ===")

try:
    result = agent.invoke({
        "messages": contact_text
    })
    
    print(f"✓ Agent response: {result['messages'][-1].content}")
    
    # In a real implementation, you would extract structured data here
    print("\n=== Simulated Structured Output ===")
    
    # Simulate extracting structured data
    structured_data = ContactInfo(
        name="John Doe",
        email="john@example.com",
        phone="(555) 123-4567",
        company="TechCorp"
    )
    
    print(f"Structured data: {structured_data}")
    print(f"\nAs JSON: {structured_data.json(indent=2)}")
    print(f"\nAs dict: {structured_data.dict()}")
    
except Exception as e:
    print(f"Error: {e}")

print("\n💡 In production, you'd get a ContactInfo object with guaranteed fields")
print("   This enables direct database insertion, API calls, etc.")